<a href="https://colab.research.google.com/github/ManG0A2/MLP-class/blob/main/3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [3]:
print(fish.shape)

(159, 6)


In [4]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [6]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy() #2차원배열로
fish_target = fish['Species'].to_numpy() #1차원배열로
print(fish_input[:5])
print(fish_target[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]
['Bream' 'Bream' 'Bream' 'Bream' 'Bream']


In [8]:
# 데이터 나누기 (테스트와 트레인)
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

In [9]:
#정규화
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [10]:
# k-최근접 알고리즘으로 예측
#결과 = 과소적합 ( 점수가 너무 낮음 )
#회귀에선 R^2이, 분류에서는 정확도가 score 로 출력 가능하다.

from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [47]:
#확인
print(kn.predict(test_scaled[:5]))

['Perch' 'Perch' 'Perch' 'Perch' 'Perch']


In [12]:
#이중분류에서는 0과 1로 분류하지만 이 모델은 다중분류이기 떄문에 위와 같은 결과가 나타난다.
#이제는 클래스 확률을 알아보자

import numpy as np

proba = kn.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=4))

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


In [14]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [ ]:
#5번째 샘플의 최근접 이웃의 클래스 확인

In [26]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [28]:
#최적의 n_neighbors 값 찾기

import matplotlib.pyplot as plt
train_score = []
test_score = []

for n_neighbors in range(3,100):
    #모델을 만든다.
    kn = KNeighborsClassifier(n_neighbors=n_neighbors)
    #모델을 훈련한다.
    kn.fit(train_scaled, train_target)
    #훈련 점수와 테스트 점수를 저장한다.
    train_score.append(kn.score(train_scaled, train_target))
    test_score.append(kn.score(test_scaled, test_target))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(3, 100), train_score, label='Train Score', marker='o')
plt.plot(range(3, 100), test_score, label='Test Score', marker='o')
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.legend()
plt.title('K-Nearest Neighbors Model Performance')
plt.grid(True)
plt.show()

In [39]:
#로지스틱 회귀
#불리언 인덱싱 (True 인 행만 선택하는 방법)

char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])

['A' 'C']


In [40]:
#도미 혹은 빙어의 행만 선택

bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
print(bream_smelt_indexes[:])

[ True False  True False False False False  True False False False  True
 False False False  True  True False False  True False  True False False
 False  True False False  True False False False False  True False False
  True  True False False False False False  True False False False False
 False  True False  True False False  True False False False  True False
 False False False False False  True False  True False False False False
 False False False False False  True False  True False False  True  True
 False False False  True False False False False False  True False False
 False  True False  True False False  True  True False False False False
 False False False False  True  True False False  True False False]


In [43]:
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

In [44]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [45]:
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [46]:
print(lr.classes_)

['Bream' 'Smelt']


In [48]:
# 무게 * x + 길이 * y + 높이 * z ~~ 같은 식으로 아래 숫자들은 계수임.
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


In [49]:
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


In [50]:
from scipy.special import expit

print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


In [51]:
#로지스틱 회귀 다중분류
#디폴트는 max_iter=100 이지만 1000으로 상향조정
#기존 선형회귀에서는 alpha 가 클수록 규제강도가 올라갔다
#하지만 여기서 매개변수는 C이고 작을수록 규제강조가 올라감

lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [53]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [56]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))
print(lr.classes_)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [57]:
#다중 분류일 경우 선형 방정식의 계수
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


이 데이터는 5개의 특성을 사용하므로 coef_ 배열의 열은 5개이다. 그런데 행이 7이다. intercept_도 7개나 있다. 이 말은 이진 분류에서 보았던 z를 7개나 계산한다는 의미이다. 혹시 눈치 챘나? 다중 분류는 클래스마다 z 값을 하나씩 계산한다. 당연히 가장 높은 z 값을 출력하는 클래스가 예측 클래스가 된다. 그럼 확률은 어떻게 계산한 것일까? 이진 분류에서는 시그모이드 함수를 사용해 z를 0과 1사이의 값으로 변환했다. 다중 분류는 이와 달리 소프트맥스 softmax 함수를 사용하여 7개의 z값을 확률로 변환한다.

In [58]:
#Z계수 혹은 함수
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [62]:
"""
모든 타겟에 대한 z를 이용한 확률 계산 (합치면 1)
axis = 1 은 각 행, 즉 각 샘플에 대해 소프트맥스를 계산
만약 axis 를 지정하지 않는다면 모든 값에 대해 소프트맥스를 계산
"""
from scipy.special import softmax
proba = softmax(decision, axis = 1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
